In [1]:
import pandas as pd
import numpy as np
import datetime

#%matplotlib inline

# Leo csv

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [4]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','descripcion','titulo','denominacion_empresa','idpais','nombre_zona','tipo_de_trabajo'],axis=1)

In [5]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [6]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Borro duplicados y agrego edad

In [7]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [8]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad                   0
dtype: int64

In [9]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)
avisos_detalle.isnull().sum()
avisos_detalle.loc[avisos_detalle['nivel_laboral'].isnull(),'nivel_laboral'] = 'Otro'
avisos_detalle.head()

,idaviso,nivel_laboral,nombre_area
0,1000872556,Senior / Semi-Senior,Producción
1,9240880,Jefe / Supervisor / Responsable,Comercial
2,1110185164,Junior,Administración
3,1110513885,Junior,Distribución
4,1111034024,Senior / Semi-Senior,Comercial


# Con esto dropeamos los avisos de postulaciones que no estan en detalles

In [10]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [11]:
#estan=postulaciones['idaviso'].isin(avisos_detalle['idaviso'])
#estan.value_counts()

In [12]:
postulaciones = postulaciones[postulaciones['idaviso'].isin(avisos_detalle['idaviso'])]

In [13]:
postulaciones.shape

(6188634, 3)

In [14]:
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Separo gente postulada y no postulada

In [15]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [16]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [17]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [18]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [19]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [20]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [21]:
gente_postulada.isnull().sum()

idpostulante          0
fechanacimiento    9983
sexo                  0
edad                  0
dtype: int64

# Arreglo el df de avisos detalle

In [22]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [23]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

In [24]:
print(avisos_detalle['nombre_area'].value_counts())

Ventas                 5256
Administración         3218
Tecnologia/Sistemas    3062
Producción             2375
Atención al Cliente    1354
Logística              1332
Oficios                1180
Recursos Humanos       1012
Finanzas                929
Ingenieria              868
Salud                   727
Gastronomia             602
Construcción            585
Otros                   583
Marketing               579
Secretaria              459
Comercio Exterior       236
Legales                 235
Minería                 124
Seguros                 106
Gerencia                102
Comunicacion             92
Educación                86
Pasantía                 70
Seguridad e Higiene      49
Química                  40
Diseño                   22
Idiomas                   5
Name: nombre_area, dtype: int64


# Funciones

In [25]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [26]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

In [27]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

# Funciones utilidad

In [28]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [29]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

# Tupla postulada y sample no postulado

In [30]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = tupla_postulada.drop(['fechanacimiento'],axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [31]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,1,48,1,0,0
1,1111920714,NM5M,1,48,1,0,0
2,1112346945,NM5M,1,48,1,0,0
3,1112345547,NM5M,1,48,1,0,0
4,1112237522,5awk,1,56,1,0,0


In [32]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],10,1000)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = sample.drop(['fechanacimiento'],axis=1)
sample = oneHotEncoding(sample,'sexo')

In [33]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,0zdxkEa,0,33,0,1,0
1,1112390041,MVrDeRN,0,27,1,0,0
2,1112390041,JBrl2wk,0,49,1,0,0
3,1112390041,VNJ8oom,0,24,0,1,0
4,1112390041,8MBwK0W,0,26,1,0,0


# Features en Test Final

In [34]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))
test_final

,idaviso,idpostulante,fechanacimiento,sexo,edad
0,739260,6M9ZQR,1976-03-29,FEM,42.0
1,739260,6v1xdL,1987-06-27,MASC,31.0
2,739260,ezRKm9,1982-02-17,FEM,36.0
3,758580,1Q35ej,1949-12-24,MASC,69.0
4,758580,EAN4J6,1986-05-27,FEM,32.0
5,758580,8R6pzR,1990-05-29,MASC,28.0
6,776420,aZJ2XN,1989-01-23,MASC,29.0
7,776420,Nmpo3J,1964-02-13,MASC,54.0
8,776420,eVqWar,1990-04-09,FEM,28.0
9,820850,6ZBD33,1980-09-18,MASC,38.0


In [35]:
test_final = feature_area_de_trabajo(test_final)

In [36]:
test_final.isnull().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo               2279
edad               2279
nivel_laboral         0
nombre_area           0
dtype: int64

In [37]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [38]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento'],axis=1)
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [39]:
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}

In [40]:
test_final['edad'] = pd.cut(test_final['edad'], range(0, 100, 5))
test_final['edad'] = test_final['edad'].astype(str)
test_final['edad'] = test_final['edad'].map(dic)

In [41]:
test_final['edad'] = test_final['edad'].fillna(2)

In [42]:
test_final.isnull().sum()

edad             0
nivel_laboral    0
nombre_area      0
FEM              0
MASC             0
NO_DECLARA       0
dtype: int64

In [43]:
test_final = oneHotEncoding(test_final,'nombre_area')
test_final = oneHotEncoding(test_final,'nivel_laboral')
#test_final = oneHotEncoding(test_final,'nombre_zona')
#test_final = oneHotEncoding(test_final,'tipo_de_trabajo')

In [44]:
test_final.isnull().sum()

edad                                    0
FEM                                     0
MASC                                    0
NO_DECLARA                              0
Administración                          0
Atención al Cliente                     0
Comercio Exterior                       0
Comunicacion                            0
Construcción                            0
Diseño                                  0
Educación                               0
Finanzas                                0
Gastronomia                             0
Gerencia                                0
Idiomas                                 0
Ingenieria                              0
Legales                                 0
Logística                               0
Marketing                               0
Minería                                 0
Oficios                                 0
Otros                                   0
Pasantía                                0
Producción                        

In [45]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,...,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,6.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,4.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,5.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,11.0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,4.0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


# Borro lo que no necesito

In [46]:
#del avisos_detalle
del postulantes_genero_y_edad
del postulaciones

# Features en el sample

In [47]:
sample = sample.append(tupla_postulada.sample(n=100000,random_state=0))

In [48]:
sample = sample.drop('idpostulante',axis=1)

In [49]:
sample['edad'] = pd.cut(sample['edad'], range(0, 100, 5))
sample['edad'] = sample['edad'].astype(str)
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
sample['edad'] = sample['edad'].map(dic)

In [50]:
sample['edad'] = sample['edad'].fillna(2)

In [51]:
sample = feature_area_de_trabajo(sample)

In [52]:
sample.isnull().sum()

idaviso          0
y                0
edad             0
FEM              0
MASC             0
NO_DECLARA       0
nivel_laboral    0
nombre_area      0
dtype: int64

## Y aca dropeamos despues

In [53]:
#sample.dropna()

In [54]:
sample = oneHotEncoding(sample,'nombre_area')
sample = oneHotEncoding(sample,'nivel_laboral')
#sample = oneHotEncoding(sample,'nombre_zona')
#sample = oneHotEncoding(sample,'tipo_de_trabajo')

In [55]:
sample = sample.drop(['idaviso'],axis=1)

In [56]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,...,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,0,4.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,3.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,7.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,2.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [57]:
sample['y'].value_counts()

1    100000
0    100000
Name: y, dtype: int64

In [58]:
test_final.count()

edad                                    100000
FEM                                     100000
MASC                                    100000
NO_DECLARA                              100000
Administración                          100000
Atención al Cliente                     100000
Comercio Exterior                       100000
Comunicacion                            100000
Construcción                            100000
Diseño                                  100000
Educación                               100000
Finanzas                                100000
Gastronomia                             100000
Gerencia                                100000
Idiomas                                 100000
Ingenieria                              100000
Legales                                 100000
Logística                               100000
Marketing                               100000
Minería                                 100000
Oficios                                 100000
Otros        

In [59]:
sample.count()

y                                       200000
edad                                    200000
FEM                                     200000
MASC                                    200000
NO_DECLARA                              200000
Administración                          200000
Atención al Cliente                     200000
Comercio Exterior                       200000
Comunicacion                            200000
Construcción                            200000
Diseño                                  200000
Educación                               200000
Finanzas                                200000
Gastronomia                             200000
Gerencia                                200000
Idiomas                                 200000
Ingenieria                              200000
Legales                                 200000
Logística                               200000
Marketing                               200000
Minería                                 200000
Oficios      

# Random Forest

In [60]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [61]:
# Divido el set para prueba local
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

In [62]:
#para el test final
train_label = []
train_data = []

train_label = sample.iloc[:, 0]
train_data = sample.iloc[:, sample.columns != 'y']

In [63]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [64]:
ModeloRFC.fit(train_data, train_label)
#ModeloRFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [65]:
result = ModeloRFC.predict(test_final)
#result = ModeloRFC.predict(X_test)

In [66]:
#from sklearn.metrics import roc_auc_score
#roc_auc_score(y_test,result)
#0.5924661884087975 con genero+edad
#0.6538586168926441 con genero+edad+nombre area
#0.6425822524525469 con genero+edad+nombre area(un poco mas reducidas-no sirvio)
#0.6570570038974716 con genero+edad+nombre area+nivel laboral
#0.668541893569122  con genero+edad+nombre area+nivel laboral+tipo trabajo+ nombre zona
#0.668541893569122  con genero+edad+nombre area+nivel laboral+tipo trabajo+ nombre zona dropeando despues
#0.6553753004113497 con genero+edad+nombre area+nivel laboral+tipo trabajo+ nombre zona sacando los avisos que no tienen detalles al ppio